In [1]:
import sys
sys.path.append('..')
import os
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, KBinsDiscretizer
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import TensorDataset, DataLoader
from torchvision.models import resnet18
import torch.nn.functional as F
import time
from sklearn.metrics import accuracy_score
from sympy import simplify_logic
from sklearn.model_selection import train_test_split

import deep_logic as dl
from deep_logic.utils.base import validate_network, set_seed, tree_to_formula
from deep_logic.utils.layer import prune_logic_layers
from deep_logic import logic

from dSprites.dSprites_loader import load_dsprites, concept_filters, get_shape_scale
from dSprites.dSprites_style_I2C import i2c_style

results_dir = './results_ll/dsprites'
if not os.path.isdir(results_dir):
    os.makedirs(results_dir)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
n_rep = 10
tot_epochs = 10001
prune_epochs = 60001

In [2]:
def get_data(path):
    # Load dataset specified by schema
    dataset_schema = 'small_skip'
    # dataset_schema = 'big_skip'

    # Get filtered concept values
    shape_range, scale_range, rot_range, x_pos_range, y_pos_range = concept_filters(dataset_schema)

    # Define function for filtering out specified concept values only
    def c_filter_fn(concepts):
        in_shape_range = (concepts[1] in shape_range)
        in_scale_range = (concepts[2] in scale_range)
        in_rot_range = (concepts[3] in rot_range)
        in_x_range = (concepts[4] in x_pos_range)
        in_y_range = (concepts[5] in y_pos_range)

        return (in_shape_range and in_scale_range and in_rot_range and in_x_range and in_y_range)

    label_fn = get_shape_scale(shape_range, scale_range)
    
    # Load dataset
    x_train, y_train, x_val, y_val, x_test, y_test, c_train, c_val, c_test, c_names = load_dsprites(path,
                                                                               c_filter_fn=c_filter_fn,
                                                                               label_fn=label_fn,
                                                                               train_test_split_flag=True)


    return x_train, y_train, x_val, y_val, x_test, y_test, c_train, c_val, c_test, c_names

# Load the model, as well as input, label, and concept data
data = get_data(path='./dSprites/data/dsprites.npz')
x_train, y_train, x_val, y_val, x_test, y_test, c_train, c_val, c_test, c_names = data
x_train = torch.FloatTensor(x_train)
y_train = torch.FloatTensor(y_train)
x_val = torch.FloatTensor(x_val)
y_val = torch.FloatTensor(y_val)
x_test = torch.FloatTensor(x_test)
y_test = torch.FloatTensor(y_test)
c_train = torch.FloatTensor(c_train)
c_val = torch.FloatTensor(c_val)
c_test = torch.FloatTensor(c_test)
print("Data loaded successfully...")

x_train shape: (20993, 3, 64, 64)
c_train shape: (20993, 50)
y_train shape: (20993, 18)
Number of images in x_train 20993
Number of images in x_val 10341
Number of images in x_test 5530
Data loaded successfully...


In [3]:
args = {
    'models_dir': './models_ll/dSprites/', 
    'model_style': 'CBM',
    'seed': 0,
    'batch_size': 128,
}

if not os.path.isdir(args['models_dir']):
    os.makedirs(args['models_dir'])

train_dataset = TensorDataset(x_train, c_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=args['batch_size'], shuffle=False)
val_dataset = TensorDataset(x_val, c_val, y_val)
valid_loader = DataLoader(val_dataset, batch_size=args['batch_size'], shuffle=False)
test_dataset = TensorDataset(x_test, c_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=args['batch_size'], shuffle=False)

In [4]:
n_concepts = c_train.size(1)
n_outputs = y_train.size(1)

In [5]:
colors = ['white']
shapes = ['square', 'ellipse', 'heart']
scale = ['very small', 'small', 's-medium', 'b-medium', 'big', 'very big']
rotation = ['0°', '5°', '10°', '15°', '20°', '25°', '30°', '35°']
x_pos = ['x0', 'x2', 'x4', 'x6', 'x8', 'x10', 'x12', 'x14', 'x16', 'x18', 'x20', 'x22', 'x24', 'x26', 'x28', 'x30']
y_pos = ['y0', 'y2', 'y4', 'y6', 'y8', 'y10', 'y12', 'y14', 'y16', 'y18', 'y20', 'y22', 'y24', 'y26', 'y28', 'y30']
concepts = colors + shapes + scale + rotation + x_pos + y_pos
len(concepts)

50

In [6]:
def cbm_learning(net_top, net_base, dataloader, device, optimizer, criterion, epoch, n_epochs, train=False):
    running_loss = 0.0
    y_correct = 0
    c_correct = 0
    c_total = 0
    y_total = 0
    total_step = len(dataloader)
    for batch_idx, (data_, concepts_, target_) in enumerate(dataloader):
        data_, concepts_, target_ = data_.to(device), concepts_.to(device), target_.to(device)

        if train:
            optimizer.zero_grad()

        c_preds = net_base(data_)
        y_preds = net_top(c_preds)
        loss = criterion(y_preds, target_) + 1 * criterion(c_preds, concepts_)

        if train:
            loss.backward()
            optimizer.step()

        running_loss += loss.item()
        c_correct += torch.sum((c_preds > 0.5).eq(concepts_)).item()
        y_correct += torch.sum((y_preds > 0.5).eq(target_)).item()
        c_total += concepts_.size(0) * concepts_.size(1)
        y_total += target_.size(0) * target_.size(1)
        if (batch_idx) % 20 == 0 and train:
            print(f'Epoch [{epoch}/{n_epochs}], Step [{batch_idx}/{total_step}], Loss: {loss.item():.4f}')

    c_accuracy = c_correct / c_total
    y_accuracy = y_correct / y_total
    loss = running_loss / total_step

    return net_top, net_base, c_accuracy, y_accuracy, loss

In [10]:
net_base = resnet18(pretrained=False)
num_ftrs = net_base.fc.in_features
net_base.fc = torch.nn.Linear(num_ftrs, n_concepts)
net_top = torch.nn.Sequential(*[
    dl.nn.XLogic(n_concepts, 32, first=True),
    torch.nn.LeakyReLU(),
    torch.nn.Linear(32, 32),
    torch.nn.LeakyReLU(),
    torch.nn.Linear(32, n_outputs),
    dl.nn.XLogic(n_outputs, n_outputs, top=True),
])
net = torch.nn.Sequential(*[net_base, net_top]).to(device)

criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.0001)

file_name = './models_ll/dsprites/trained_model.pt'

# if os.path.isfile(file_name):
#     # load trained model
#     model.load_state_dict(torch.load(file_name))

# else:
set_seed(0)
need_pruning = True

n_epochs = 200
valid_loss_min = np.Inf
val_loss = []
val_acc = []
val_c_acc = []
test_loss = []
test_acc = []
test_c_acc = []
train_loss = []
train_acc = []
train_c_acc = []
epoch = 1
for epoch in range(1, n_epochs + 1):
    print(f'Epoch {epoch}\n')

    net_top, net_base, c_accuracy, y_accuracy, loss = cbm_learning(net_top, net_base, train_loader,
                                                               device, optimizer, criterion,
                                                               epoch, n_epochs, train=True)
    train_c_acc.append(c_accuracy)
    train_acc.append(y_accuracy)
    train_loss.append(loss)
    print(f'\ntrain-loss: {loss:.4f}, train-c-acc: {c_accuracy:.4f}, train-cy-acc: {y_accuracy:.4f}')

    with torch.no_grad():
        net_top.eval()
        net_base.eval()

        net_top, net_base, c_accuracy, y_accuracy, loss = cbm_learning(net_top, net_base, valid_loader,
                                                                   device, optimizer, criterion,
                                                                   epoch, n_epochs, train=False)
        val_c_acc.append(c_accuracy)
        val_acc.append(y_accuracy)
        val_loss.append(loss)
        print(f'validation loss: {loss:.4f}, validation-c-acc: {c_accuracy:.4f}, validation-y-acc: {y_accuracy:.4f}')

        if loss < valid_loss_min:
            valid_loss_min = loss
            torch.save(net.state_dict(), file_name)
            print('Improvement-Detected, save-model')

        net_top, net_base, c_accuracy, y_accuracy, loss = cbm_learning(net_top, net_base, test_loader,
                                                                   device, optimizer, criterion,
                                                                   epoch, n_epochs, train=False)
        test_c_acc.append(c_accuracy)
        test_acc.append(y_accuracy)
        test_loss.append(loss)
        print(f'test loss: {loss:.4f}, test-c-acc: {c_accuracy:.4f}, test-y-acc: {y_accuracy:.4f}\n')

    net_top.train()
    net_base.train()

results = pd.DataFrame({
    'test_acc': test_acc,
    'test_c_acc': test_c_acc,
    'test_loss': test_loss,
    'val_acc': val_acc,
    'val_c_acc': val_c_acc,
    'val_loss': val_loss,
    'train_acc': train_acc,
    'train_c_acc': train_c_acc,
    'train_loss': train_loss,
})

Epoch 1

Epoch [1/200], Step [0/165], Loss: 0.6724
Epoch [1/200], Step [20/165], Loss: 0.3396
Epoch [1/200], Step [40/165], Loss: 0.3230
Epoch [1/200], Step [60/165], Loss: 0.3168
Epoch [1/200], Step [80/165], Loss: 0.3105
Epoch [1/200], Step [100/165], Loss: 0.3080
Epoch [1/200], Step [120/165], Loss: 0.3026
Epoch [1/200], Step [140/165], Loss: 0.2978
Epoch [1/200], Step [160/165], Loss: 0.2932

train-loss: 0.3211, train-c-acc: 0.9034, train-cy-acc: 0.6325
validation loss: 0.2894, validation-c-acc: 0.9158, validation-y-acc: 0.7452
Improvement-Detected, save-model
test loss: 0.2896, test-c-acc: 0.9153, test-y-acc: 0.7449

Epoch 2

Epoch [2/200], Step [0/165], Loss: 0.2876
Epoch [2/200], Step [20/165], Loss: 0.2805
Epoch [2/200], Step [40/165], Loss: 0.2694
Epoch [2/200], Step [60/165], Loss: 0.2574
Epoch [2/200], Step [80/165], Loss: 0.2408
Epoch [2/200], Step [100/165], Loss: 0.2284
Epoch [2/200], Step [120/165], Loss: 0.2145
Epoch [2/200], Step [140/165], Loss: 0.1989
Epoch [2/200], 

Epoch [14/200], Step [20/165], Loss: 0.0623
Epoch [14/200], Step [40/165], Loss: 0.0630
Epoch [14/200], Step [60/165], Loss: 0.0621
Epoch [14/200], Step [80/165], Loss: 0.0616
Epoch [14/200], Step [100/165], Loss: 0.0630
Epoch [14/200], Step [120/165], Loss: 0.0623
Epoch [14/200], Step [140/165], Loss: 0.0612
Epoch [14/200], Step [160/165], Loss: 0.0619

train-loss: 0.0621, train-c-acc: 0.9948, train-cy-acc: 0.9444
validation loss: 0.0752, validation-c-acc: 0.9795, validation-y-acc: 0.9444
test loss: 0.0750, test-c-acc: 0.9799, test-y-acc: 0.9444

Epoch 15

Epoch [15/200], Step [0/165], Loss: 0.0616
Epoch [15/200], Step [20/165], Loss: 0.0612
Epoch [15/200], Step [40/165], Loss: 0.0619
Epoch [15/200], Step [60/165], Loss: 0.0609
Epoch [15/200], Step [80/165], Loss: 0.0611
Epoch [15/200], Step [100/165], Loss: 0.0624
Epoch [15/200], Step [120/165], Loss: 0.0607
Epoch [15/200], Step [140/165], Loss: 0.0611
Epoch [15/200], Step [160/165], Loss: 0.0611

train-loss: 0.0611, train-c-acc: 0.9

Epoch [27/200], Step [100/165], Loss: 0.0396
Epoch [27/200], Step [120/165], Loss: 0.0395
Epoch [27/200], Step [140/165], Loss: 0.0390
Epoch [27/200], Step [160/165], Loss: 0.0402

train-loss: 0.0397, train-c-acc: 0.9991, train-cy-acc: 0.9615
validation loss: 0.0529, validation-c-acc: 0.9861, validation-y-acc: 0.9552
Improvement-Detected, save-model
test loss: 0.0530, test-c-acc: 0.9861, test-y-acc: 0.9550

Epoch 28

Epoch [28/200], Step [0/165], Loss: 0.0366
Epoch [28/200], Step [20/165], Loss: 0.0380
Epoch [28/200], Step [40/165], Loss: 0.0363
Epoch [28/200], Step [60/165], Loss: 0.0356
Epoch [28/200], Step [80/165], Loss: 0.0370
Epoch [28/200], Step [100/165], Loss: 0.0362
Epoch [28/200], Step [120/165], Loss: 0.0362
Epoch [28/200], Step [140/165], Loss: 0.0355
Epoch [28/200], Step [160/165], Loss: 0.0371

train-loss: 0.0366, train-c-acc: 0.9992, train-cy-acc: 0.9640
validation loss: 0.0489, validation-c-acc: 0.9874, validation-y-acc: 0.9633
Improvement-Detected, save-model
test los

Epoch [40/200], Step [120/165], Loss: 0.0084
Epoch [40/200], Step [140/165], Loss: 0.0070
Epoch [40/200], Step [160/165], Loss: 0.0073

train-loss: 0.0084, train-c-acc: 0.9996, train-cy-acc: 0.9998
validation loss: 0.0232, validation-c-acc: 0.9889, validation-y-acc: 0.9935
Improvement-Detected, save-model
test loss: 0.0230, test-c-acc: 0.9893, test-y-acc: 0.9937

Epoch 41

Epoch [41/200], Step [0/165], Loss: 0.0076
Epoch [41/200], Step [20/165], Loss: 0.0076
Epoch [41/200], Step [40/165], Loss: 0.0078
Epoch [41/200], Step [60/165], Loss: 0.0068
Epoch [41/200], Step [80/165], Loss: 0.0075
Epoch [41/200], Step [100/165], Loss: 0.0077
Epoch [41/200], Step [120/165], Loss: 0.0072
Epoch [41/200], Step [140/165], Loss: 0.0067
Epoch [41/200], Step [160/165], Loss: 0.0069

train-loss: 0.0074, train-c-acc: 0.9996, train-cy-acc: 0.9999
validation loss: 0.0287, validation-c-acc: 0.9869, validation-y-acc: 0.9862
test loss: 0.0287, test-c-acc: 0.9870, test-y-acc: 0.9863

Epoch 42

Epoch [42/200], S

test loss: 0.0185, test-c-acc: 0.9897, test-y-acc: 0.9940

Epoch 54

Epoch [54/200], Step [0/165], Loss: 0.0073
Epoch [54/200], Step [20/165], Loss: 0.0052
Epoch [54/200], Step [40/165], Loss: 0.0043
Epoch [54/200], Step [60/165], Loss: 0.0035
Epoch [54/200], Step [80/165], Loss: 0.0033
Epoch [54/200], Step [100/165], Loss: 0.0033
Epoch [54/200], Step [120/165], Loss: 0.0032
Epoch [54/200], Step [140/165], Loss: 0.0034
Epoch [54/200], Step [160/165], Loss: 0.0030

train-loss: 0.0038, train-c-acc: 0.9997, train-cy-acc: 0.9997
validation loss: 0.0165, validation-c-acc: 0.9924, validation-y-acc: 0.9917
Improvement-Detected, save-model
test loss: 0.0167, test-c-acc: 0.9921, test-y-acc: 0.9915

Epoch 55

Epoch [55/200], Step [0/165], Loss: 0.0032
Epoch [55/200], Step [20/165], Loss: 0.0033
Epoch [55/200], Step [40/165], Loss: 0.0029
Epoch [55/200], Step [60/165], Loss: 0.0024
Epoch [55/200], Step [80/165], Loss: 0.0027
Epoch [55/200], Step [100/165], Loss: 0.0025
Epoch [55/200], Step [120/1

...

Epoch [147/200], Step [100/165], Loss: 0.0064
Epoch [147/200], Step [120/165], Loss: 0.0028
Epoch [147/200], Step [140/165], Loss: 0.0029
Epoch [147/200], Step [160/165], Loss: 0.0095

train-loss: 0.0078, train-c-acc: 0.9977, train-cy-acc: 0.9960
validation loss: 0.0127, validation-c-acc: 0.9930, validation-y-acc: 0.9959
test loss: 0.0126, test-c-acc: 0.9929, test-y-acc: 0.9960

Epoch 148

Epoch [148/200], Step [0/165], Loss: 0.0114
Epoch [148/200], Step [20/165], Loss: 0.0049
Epoch [148/200], Step [40/165], Loss: 0.0018
Epoch [148/200], Step [60/165], Loss: 0.0024
Epoch [148/200], Step [80/165], Loss: 0.0015
Epoch [148/200], Step [100/165], Loss: 0.0024
Epoch [148/200], Step [120/165], Loss: 0.0031
Epoch [148/200], Step [140/165], Loss: 0.0013
Epoch [148/200], Step [160/165], Loss: 0.0016

train-loss: 0.0024, train-c-acc: 0.9995, train-cy-acc: 0.9992
validation loss: 0.0043, validation-c-acc: 0.9976, validation-y-acc: 0.9995
Improvement-Detected, save-model
test loss: 0.0041, test-c-a

Epoch [160/200], Step [140/165], Loss: 0.0004
Epoch [160/200], Step [160/165], Loss: 0.0004

train-loss: 0.0003, train-c-acc: 1.0000, train-cy-acc: 1.0000
validation loss: 0.0051, validation-c-acc: 0.9971, validation-y-acc: 0.9990
test loss: 0.0052, test-c-acc: 0.9970, test-y-acc: 0.9988

Epoch 161

Epoch [161/200], Step [0/165], Loss: 0.0004
Epoch [161/200], Step [20/165], Loss: 0.0003
Epoch [161/200], Step [40/165], Loss: 0.0003
Epoch [161/200], Step [60/165], Loss: 0.0003
Epoch [161/200], Step [80/165], Loss: 0.0003
Epoch [161/200], Step [100/165], Loss: 0.0003
Epoch [161/200], Step [120/165], Loss: 0.0003
Epoch [161/200], Step [140/165], Loss: 0.0004
Epoch [161/200], Step [160/165], Loss: 0.0004

train-loss: 0.0003, train-c-acc: 1.0000, train-cy-acc: 1.0000
validation loss: 0.0066, validation-c-acc: 0.9960, validation-y-acc: 0.9983
test loss: 0.0067, test-c-acc: 0.9958, test-y-acc: 0.9982

Epoch 162

Epoch [162/200], Step [0/165], Loss: 0.0004
Epoch [162/200], Step [20/165], Loss: 

test loss: 0.0058, test-c-acc: 0.9969, test-y-acc: 0.9988

Epoch 174

Epoch [174/200], Step [0/165], Loss: 0.0008
Epoch [174/200], Step [20/165], Loss: 0.0007
Epoch [174/200], Step [40/165], Loss: 0.0010
Epoch [174/200], Step [60/165], Loss: 0.0008
Epoch [174/200], Step [80/165], Loss: 0.0006
Epoch [174/200], Step [100/165], Loss: 0.0007
Epoch [174/200], Step [120/165], Loss: 0.0006
Epoch [174/200], Step [140/165], Loss: 0.0005
Epoch [174/200], Step [160/165], Loss: 0.0005

train-loss: 0.0007, train-c-acc: 0.9999, train-cy-acc: 1.0000
validation loss: 0.0069, validation-c-acc: 0.9969, validation-y-acc: 0.9979
test loss: 0.0068, test-c-acc: 0.9969, test-y-acc: 0.9978

Epoch 175

Epoch [175/200], Step [0/165], Loss: 0.0011
Epoch [175/200], Step [20/165], Loss: 0.0008
Epoch [175/200], Step [40/165], Loss: 0.0010
Epoch [175/200], Step [60/165], Loss: 0.0042
Epoch [175/200], Step [80/165], Loss: 0.0152
Epoch [175/200], Step [100/165], Loss: 0.0243
Epoch [175/200], Step [120/165], Loss: 0.00

Epoch [187/200], Step [60/165], Loss: 0.0005
Epoch [187/200], Step [80/165], Loss: 0.0005
Epoch [187/200], Step [100/165], Loss: 0.0005
Epoch [187/200], Step [120/165], Loss: 0.0006
Epoch [187/200], Step [140/165], Loss: 0.0004
Epoch [187/200], Step [160/165], Loss: 0.0004

train-loss: 0.0004, train-c-acc: 1.0000, train-cy-acc: 1.0000
validation loss: 0.0049, validation-c-acc: 0.9968, validation-y-acc: 0.9995
test loss: 0.0049, test-c-acc: 0.9965, test-y-acc: 0.9995

Epoch 188

Epoch [188/200], Step [0/165], Loss: 0.0005
Epoch [188/200], Step [20/165], Loss: 0.0004
Epoch [188/200], Step [40/165], Loss: 0.0004
Epoch [188/200], Step [60/165], Loss: 0.0005
Epoch [188/200], Step [80/165], Loss: 0.0005
Epoch [188/200], Step [100/165], Loss: 0.0005
Epoch [188/200], Step [120/165], Loss: 0.0006
Epoch [188/200], Step [140/165], Loss: 0.0003
Epoch [188/200], Step [160/165], Loss: 0.0004

train-loss: 0.0004, train-c-acc: 1.0000, train-cy-acc: 1.0000
validation loss: 0.0045, validation-c-acc: 0.9

Epoch [200/200], Step [140/165], Loss: 0.0004
Epoch [200/200], Step [160/165], Loss: 0.0005

train-loss: 0.0005, train-c-acc: 1.0000, train-cy-acc: 1.0000
validation loss: 0.0035, validation-c-acc: 0.9979, validation-y-acc: 0.9997
Improvement-Detected, save-model
test loss: 0.0035, test-c-acc: 0.9980, test-y-acc: 0.9996

